# 01: Quickstart — From Zero to Enforced Architecture

You have a TypeScript project. It compiles fine, but nothing stops a developer from importing `fs` in the domain layer or calling the database from a controller.

This notebook walks through adding KindScript enforcement:

**Part 1 — Define and enforce:** Write a `.k.ts` definition file, run `ksc check`

**Part 2 — Advanced:** Standalone members, nested structure, path overrides

**Prerequisites:** Run `npx tsc` from the project root to compile the CLI.

## Setup

In [ ]:
import { PROJECT_ROOT, KSC, ksc, tree, copyFixture, writeFile, readFile, cleanup } from './lib.ts';
console.log("CLI path:", KSC);

---

## Part 1: Define Architecture and Enforce Contracts

We'll create a small Clean Architecture project, write the architectural definitions, and enforce contracts with `ksc check`.

### Step 0: Create a realistic project

In [ ]:
const DEMO = copyFixture("quickstart");
console.log("Demo project:", DEMO);

console.log("\nProject structure:");
await tree(DEMO);

### Step 1: Write architecture definitions

Create `src/context.k.ts` — the source of truth for your architectural rules. Definition files use the `.k.ts` extension and are auto-discovered by KindScript. The root directory is inferred from the `.k.ts` file's location.

It defines Kind types with constraints, and uses `satisfies InstanceConfig<T>` to map them to filesystem locations.

In [ ]:
writeFile(DEMO, "src/context.k.ts", `
import type { Kind, ConstraintConfig, InstanceConfig } from 'kindscript';

// Kind definitions — normative (what the architecture MUST look like)
type DomainLayer = Kind<"DomainLayer">;
type ApplicationLayer = Kind<"ApplicationLayer">;
type InfrastructureLayer = Kind<"InfrastructureLayer">;

type CleanArchitectureContext = Kind<"CleanArchitectureContext", {
  domain: DomainLayer;
  application: ApplicationLayer;
  infrastructure: InfrastructureLayer;
}, {
  noDependency: [["domain", "infrastructure"], ["domain", "application"]];
}>;

// Instance — maps the Kind type to real directories on disk
export const app = {
  domain: {},
  application: {},
  infrastructure: {},
} satisfies InstanceConfig<CleanArchitectureContext>;
`);

console.log("=== src/context.k.ts ===");
console.log(readFile(DEMO, "src/context.k.ts"));

The `src/context.k.ts` has two sections:

1. **Kind definitions** — type aliases using `Kind<N>` and `Kind<N, Members, Constraints>`. `CleanArchitectureContext` is a composite that groups `DomainLayer`, `ApplicationLayer`, and `InfrastructureLayer` as members. Constraints (like `noDependency`) are declared as the third type parameter.
2. **Instance declaration** — `{ ... } satisfies InstanceConfig<CleanArchitectureContext>` maps each member name to a directory. The root is inferred from the `.k.ts` file's directory (`src/`), so member name `domain` → `src/domain/`, `application` → `src/application/`, etc.

All KindScript APIs are **pure types** — zero runtime footprint. The `satisfies` operator validates the object literal shape at compile time, and the KindScript compiler reads it from the AST.

No `kindscript.json` is needed — `.k.ts` files are auto-discovered.

### Step 2: Check contracts — should pass

In [ ]:
await ksc("check", DEMO);

### Step 3: Break a rule and see it caught

The domain layer should never depend on infrastructure. Let's add a forbidden import.

In [ ]:
writeFile(DEMO, "src/domain/user.ts", `
import { saveUser } from '../infrastructure/user-repo';

export interface User {
  id: string;
  name: string;
  email: string;
}

export function createUser(name: string, email: string): User {
  const user = { id: Math.random().toString(36), name, email };
  saveUser(user); // BAD: domain reaching into infrastructure
  return user;
}
`);

console.log("Violation introduced: domain/user.ts now imports from infrastructure");

In [ ]:
await ksc("check", DEMO);

KindScript caught it. The error tells you:
- **Where**: `src/domain/user.ts` line 1
- **What**: Forbidden dependency — domain is importing from infrastructure
- **Why**: The `noDependency(domain -> infrastructure)` contract
- **Error code**: `KS70001`

Exit code 1 means CI pipelines fail automatically.

### Step 4: Fix and verify

In [ ]:
writeFile(DEMO, "src/domain/user.ts", `
export interface User {
  id: string;
  name: string;
  email: string;
}

export function createUser(name: string, email: string): User {
  return { id: Math.random().toString(36), name, email };
}
`);

await ksc("check", DEMO);

cleanup(DEMO);
console.log("Demo project cleaned up.");

---

## Part 2: Advanced — Standalone Members, Nesting, and Path Overrides

For larger architectures, `InstanceConfig<T>` supports three advanced features.

### Standalone Members

Members can be defined as standalone typed constants, then composed into the parent. This improves readability for complex hierarchies.

```typescript
// Define members separately
const domain: DomainLayer = {
  entities: {},
  ports: {},
};

// Compose into the instance
export const app = {
  domain,           // standalone reference
  application: {},  // inline (fine for leaves)
  infrastructure: {},
} satisfies InstanceConfig<AppContext>;
```

The classifier resolves variable references — `domain` is looked up, its type annotation provides the Kind identity, and its initializer provides sub-member assignments.

### Nested Structure

Kind definitions can have nested sub-members. Path derivation is recursive:

```typescript
type EntitiesModule = Kind<"EntitiesModule">;
type PortsModule = Kind<"PortsModule">;

type DomainLayer = Kind<"DomainLayer", {
  entities: EntitiesModule;
  ports: PortsModule;
}>;
```

With the `.k.ts` file in `src/`, the derived paths are:
- `domain` → `src/domain/`
- `domain.entities` → `src/domain/entities/`
- `domain.ports` → `src/domain/ports/`

### Path Overrides

By default, the member name is the directory name. If your filesystem uses a different convention, override per-member:

```typescript
const domain: DomainLayer = {
  valueObjects: { path: "value-objects" },  // src/domain/value-objects/ instead of src/domain/valueObjects/
  entities: {},
};
```

Override is per-member, per-instance. The Kind definition stays clean.

### Multi-Instance (Bounded Contexts)

The same Kind definition can be instantiated multiple times with separate `.k.ts` files. Each `.k.ts` file's directory becomes the root:

```
src/
  ordering/
    ordering.k.ts          # root = src/ordering/
    domain/
    application/
    infrastructure/
  billing/
    billing.k.ts           # root = src/billing/
    domain/
    application/
    infrastructure/
```

`ordering.domain` resolves to `src/ordering/domain`. `billing.domain` resolves to `src/billing/domain`. Same structure, different positions.

---

## Summary

| Command | What it does |
|---------|-------------|
| `ksc check` | Validates all contracts — exit 0 if clean, exit 1 if violations |

**Typical workflows:**
- **Existing project**: write `src/context.k.ts` → `ksc check`
- **New project**: write `src/context.k.ts` → create directories → `ksc check`
- **CI pipeline**: `ksc check` on every PR

**Next:** See **02-contracts.ipynb** for all 6 contract types